In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *

In [0]:
df=spark.read.load('/FileStore/tables/googleplaystore.csv',format='csv',sep=',',header='true',escape='"',inferschema='true')

In [0]:
df.count()

10841

In [0]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [0]:
# Data Cleaning Step 

df=df.drop("size","Content Rating","Last Updated","Android Ver")

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Current Ver: string (nullable = true)



In [0]:
df=df.drop("Current Ver")

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
df=df.withColumn('Reviews',col('Reviews').cast(IntegerType()))\
.withColumn('installs',regexp_replace(col('Installs'),'[^0-9]',''))\
.withColumn('Installs',col('Installs').cast(IntegerType()))\
.withColumn('Price',regexp_replace(col('Price'),'[$]',''))\
.withColumn('Price',col('Price').cast(IntegerType()))

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [0]:
df.createOrReplaceTempView("apps")

In [0]:
%sql 
select count(*) from apps

count(1)
10841


In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
%sql

select App,sum(Reviews) from apps group by 1 order by 2 desc LIMIT 10;

App,sum(Reviews)
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958
Candy Crush Saga,156993136
Facebook,156286514
8 Ball Pool,99386198
Clash Royale,92530298
Snapchat,68045010


In [0]:
%sql select app,sum(installs) as Total_Installs ,Type from apps group by 1,3 order by 2 desc limit 10;

app,Total_Installs,Type
Subway Surfers,6000000000,Free
Instagram,4000000000,Free
Google Drive,4000000000,Free
Hangouts,4000000000,Free
Google Photos,4000000000,Free
Google News,4000000000,Free
Candy Crush Saga,3500000000,Free
WhatsApp Messenger,3000000000,Free
Gmail,3000000000,Free
Temple Run 2,3000000000,Free


In [0]:
%sql
select Category, sum(Installs) as Total_Installs from apps group by 1 order by 2 desc ;

Category,Total_Installs
GAME,35086024415
COMMUNICATION,32647276251
PRODUCTIVITY,14176091369
SOCIAL,14069867902
TOOLS,11452771915
FAMILY,10258263505
PHOTOGRAPHY,10088247655
NEWS_AND_MAGAZINES,7496317760
TRAVEL_AND_LOCAL,6868887146
VIDEO_PLAYERS,6222002720


In [0]:
%sql 
SELECT app, SUM(Installs) AS Total_installs 
FROM apps 
WHERE Type='Paid' 
GROUP BY app 
ORDER BY Total_installs DESC;

app,Total_installs
Minecraft,20000000
Hitman Sniper,10000000
Facetune - For Free,3000000
Beautiful Widgets Pro,2000000
HD Widgets,2000000
League of Stickman 2018- Ninja Arena PVP(Dreamsky),1000000
Fruit Ninja Classic,1000000
Toca Life: City,1000000
DraStic DS Emulator,1000000
Five Nights at Freddy's,1000000


In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
%sql 
SELECT app, SUM(Reviews) AS Top_Reviews 
FROM apps 
WHERE Type='Paid' 
GROUP BY app 
ORDER BY Top_Reviews DESC;

app,Top_Reviews
Minecraft,4751900
Hitman Sniper,408292
Grand Theft Auto: San Andreas,348962
Beautiful Widgets Pro,195780
Bloons TD 5,190086
Where's My Water?,188740
Facetune - For Free,148659
Card Wars - Adventure Time,129603
True Skate,129409
HD Widgets,117231
